In [1]:
import torch
from torch import nn
from src.utils import load_model
from src.data_loader import get_cifar10_loader
from src.train import train_model_kd, loss_fn_kd
from src.model import ResNet, BasicBlock
from src.evaluate import evaluate
from src.train import KDParams


In [5]:
# Parameters
device = torch.device("mps")
teacher_model_path = "models/resnet110_baseline_30_mps.pth"
student_model_path = "models/pruned_95-10_resnet110_mps.pth"

batch_size = 128
learning_rate = 0.001
num_epochs = 10
kd_alpha = 0.7
kd_temperature = 4.0


In [6]:
# Load pretrained ResNet model
teacher_model = load_model(teacher_model_path, device=device)
student_model = load_model(student_model_path, device=device)

# Define optimizer and criterion for training
optimizer = torch.optim.Adam(student_model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Load data
train_loader = get_cifar10_loader('train', batch_size=batch_size)
val_loader = get_cifar10_loader('val', batch_size=batch_size)

kd_params = KDParams(alpha=kd_alpha, temperature=kd_temperature)


In [7]:
train_model_kd(
    student_model=student_model,
    teacher_model=teacher_model,
    train_loader=train_loader,
    optimizer=optimizer,
    device=device,
    kd_params=kd_params,
    num_epochs=num_epochs,
    loss_fn_kd=loss_fn_kd,
    use_amp=False
)


In [8]:
evaluate(student_model, val_loader, device)

Validation Accuracy: 57.75%, Avg Loss: 1.3852, Time: 2.97s


(57.75, 1.385177766418457)